In [ ]:
import numpy as np

In [13]:
# 特征化练习1--提取字典类数据
# 使用DictVectorizer
from sklearn.feature_extraction import DictVectorizer

def dictvec():
    # 字典类的数据
    dict_data=[{'fruit':'apple','price':5,'discount':['1','2','3']},
               {'fruit':'orange','price':3,'discount':['2','3']},
               {'fruit':'pear','price':6,'discount':['3']}]
    # 创建一个DictVectorizer类,创建时要指出是否进行压缩存储
    # 默认sparse为True即不进行压缩存储
    dict=DictVectorizer(sparse=False)

    # 通过fit_transform来讲字典类数据进行转换
    data=dict.fit_transform(dict_data)
    # 打印查看转换效果
    print(data)
    print('-'*50)
    # 通过get_feature_name_out获取转换后数据的特征类型有哪些
    print(dict.get_feature_names_out())
    print('-'*50)
    # 通过inverse_transform(data),将特征逆转回原数据,知晓特征所指的具体含义
    print(dict.inverse_transform(data))

dictvec()

[[1. 1. 1. 1. 0. 0. 5.]
 [0. 1. 1. 0. 1. 0. 3.]
 [0. 0. 1. 0. 0. 1. 6.]]
--------------------------------------------------
['discount=1' 'discount=2' 'discount=3' 'fruit=apple' 'fruit=orange'
 'fruit=pear' 'price']
--------------------------------------------------
[{'discount=1': 1.0, 'discount=2': 1.0, 'discount=3': 1.0, 'fruit=apple': 1.0, 'price': 5.0}, {'discount=2': 1.0, 'discount=3': 1.0, 'fruit=orange': 1.0, 'price': 3.0}, {'discount=3': 1.0, 'fruit=pear': 1.0, 'price': 6.0}]


转换规则总结:<br>
字典类的键值对会形成一个特征,若键值均为字符串,那么二者合二为一成为一个特征记录,以one-hot编码记录<br>
若键为字符串,值为数值型,则键作为一种特征,对应特征数据就是值<br>
若值为列表型,列表内只能为字符串类型,并且键会与列表内的值一一对应组成一个特征类型

In [16]:
# 英文文本特征抽取
# 使用CountVectorizer进行简单的计数再提取
from sklearn.feature_extraction.text import CountVectorizer
def countvec():
    # 实例化CountVectorizer
    # 默认单个字母的单词不计入,默认认为这个词对整个样本无影响
    # 在实例化时也可以传入参数max_df或min_df,规定单词的出现次数来对计数后的单词进行删选
    count=CountVectorizer()
    # 英文文本数据
    text=["hello world, hello python, i love python",
          "hello python, i love the world forever",
          "hello programe, i love programe"]
    # 通过fit_transform来对数据进行转换
    data=count.fit_transform(text)

    # 输出查看处理效果
    # 默认输出的形式是按压缩矩阵的形式输出
    print(data)
    print('-'*50)
    # 通过toarray更方便的查看数据
    print(data.toarray())
    print('-'*50)
    # 通过get_feature_name_out获取转换后数据的特征类型有哪些
    print(count.get_feature_names_out())
    print('-'*50)
    # 通过inverse_transform(data),将特征逆转回原数据,知晓特征所指的具体含义
    print(count.inverse_transform(data))

countvec()

  (0, 1)	2
  (0, 6)	1
  (0, 4)	2
  (0, 2)	1
  (1, 1)	1
  (1, 6)	1
  (1, 4)	1
  (1, 2)	1
  (1, 5)	1
  (1, 0)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2
--------------------------------------------------
[[0 2 1 0 2 0 1]
 [1 1 1 0 1 1 1]
 [0 1 1 2 0 0 0]]
--------------------------------------------------
['forever' 'hello' 'love' 'programe' 'python' 'the' 'world']
--------------------------------------------------
[array(['hello', 'world', 'python', 'love'], dtype='<U8'), array(['hello', 'world', 'python', 'love', 'the', 'forever'], dtype='<U8'), array(['hello', 'love', 'programe'], dtype='<U8')]


规则总结:<br>
CountVectorizer是通过空格标点等将英文文本分割为一个个独立的单词,并对单词进行计数<br>
统计除单个字母单词之外的所有单词作为特征类型,输出每一个样本中所含对应特征类型的次数

In [18]:
# 中文文本处理
from sklearn.feature_extraction.text import CountVectorizer
# 汉字文本特征化,CountVectorizer进行空格分割汉字不太合理
def countvec():
    """
    对文本进行特征值化,单个汉字单个字母不统计，因为单个汉字字母没有意义
    :return: None
    """
    cv = CountVectorizer()

    data = cv.fit_transform(["人生苦短，我 喜欢 python python", "人生漫长，不用 python"])

    print(cv.get_feature_names_out())

    print(data)
    print(data.toarray())

    return None

countvec()

['python' '不用' '人生漫长' '人生苦短' '喜欢']
  (0, 3)	1
  (0, 4)	1
  (0, 0)	2
  (1, 0)	1
  (1, 2)	1
  (1, 1)	1
[[2 0 0 1 1]
 [1 1 1 0 0]]


对于中文文本而言,CountVectorizer自带的分割逻辑不太适用<br>
这时候可以导入jieba的中文分词包来辅助处理,先利用包进行分词后再计数转换

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
import jieba
def cut_word():
    # 通过jieba.cut来对中文文本进行更加合理的分词
    # cut分词后得到是一个生成器类
    con1 = jieba.cut("今天很残酷，明天更残酷，后天很美好，但绝对大部分是死在明天晚上，所以每个人不要放弃今天。")

    con2 = jieba.cut("我们看到的从很远星系来的光是在几百万年之前发出的，这样当我们看到宇宙时，我们是在看它的过去。")

    con3 = jieba.cut("如果只用一种方式了解某样事物，你就不会真正了解它。了解事物真正含义的秘密取决于如何将其与我们所了解的事物相联系。")

    # 先将生成器转为列表来查看分词效果,且利于下部操作
    con1=list(con1)
    con2=list(con2)
    con3=list(con3)
    print(len(con1),len(con2),len(con3))
    print(con1)
    print(con2)
    print(con3)
    print('-'*50)
    # 转为list后的另一个好处就在于,可以通过str.join(list)
    # 将切分好的中文文本列表通姑空格 连接起来,这样就可以完美符合CountVectorizer按空格分词的特性进行特征化
    text1=' '.join(con1)
    text2=' '.join(con2)
    text3=' '.join(con3)
    return text1,text2,text3

def chinese_text_vec():
    # 对结巴分词后的中文文本进行转换
    # 接受数据
    t1,t2,t3=cut_word()
    # 创建类
    cv = CountVectorizer()
    # 转换数据，提取特征
    data = cv.fit_transform([t1,t2,t3])
    # 输出结果
    print(cv.get_feature_names_out())
    print(data)
    print(data.toarray())

chinese_text_vec()

28 30 34
['今天', '很', '残酷', '，', '明天', '更', '残酷', '，', '后天', '很', '美好', '，', '但', '绝对', '大部分', '是', '死', '在', '明天', '晚上', '，', '所以', '每个', '人', '不要', '放弃', '今天', '。']
['我们', '看到', '的', '从', '很', '远', '星系', '来', '的', '光是在', '几百万年', '之前', '发出', '的', '，', '这样', '当', '我们', '看到', '宇宙', '时', '，', '我们', '是', '在', '看', '它', '的', '过去', '。']
['如果', '只用', '一种', '方式', '了解', '某样', '事物', '，', '你', '就', '不会', '真正', '了解', '它', '。', '了解', '事物', '真正', '含义', '的', '秘密', '取决于', '如何', '将', '其', '与', '我们', '所', '了解', '的', '事物', '相', '联系', '。']
--------------------------------------------------
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
  (0, 6)	2
  (0, 26)	2
  (0, 22)	2
  (0, 12)	1
  (0, 32)	1
  (0, 31)	1
  (0, 14)	1
  (0, 24)	1
  (0, 19)	1
  (0, 27)	1
  (0, 2)	1
  (0, 20)	1
  (1, 18)	3
  (1, 28)	2
  (1, 23)	1
  (1, 7)	1
  (1, 8)	1
  (1, 3)	1
  (1, 9)	1
  (1, 35)	1


### TF-IDF判定法
通过计算中文文本转化后特征的tf-idf值，来判定这个特征的重要性<br>
tf值是值该特征词在文本中的出现频率<br>
idf值叫做逆文档频率，即分析一个词在多少个文本有出现，若出现次数越多，idf值越小<br>
计算规则为idf=lg（D/j),D为文本总数，j为包含该词的文本数

In [35]:
# 计算上述中文文本特征化后的tf-idf值
from sklearn.feature_extraction.text import TfidfVectorizer
def tfivec():
    # 接收切割好的文本数据
    t1,t2,t3=cut_word()
    # 创建类
    # 创建过程中可传入一定的参数，如smooth_idf,默认为True，即对计算后的数据进行一定的平滑性处理
    # 旨在针对与出现次数为0的词进行处理，计算规则处理为idf=lg（D+1/j+1) + 1
    tfi=TfidfVectorizer()
    # 转换数据，计算tfidf值
    data=tfi.fit_transform([t1,t2,t3])
    # 输出结果
    print(tfi.get_feature_names_out())
    print('-'*50)
    print(data)
    print('-'*50)
    print(data.toarray())

tfivec()

28 30 34
['今天', '很', '残酷', '，', '明天', '更', '残酷', '，', '后天', '很', '美好', '，', '但', '绝对', '大部分', '是', '死', '在', '明天', '晚上', '，', '所以', '每个', '人', '不要', '放弃', '今天', '。']
['我们', '看到', '的', '从', '很', '远', '星系', '来', '的', '光是在', '几百万年', '之前', '发出', '的', '，', '这样', '当', '我们', '看到', '宇宙', '时', '，', '我们', '是', '在', '看', '它', '的', '过去', '。']
['如果', '只用', '一种', '方式', '了解', '某样', '事物', '，', '你', '就', '不会', '真正', '了解', '它', '。', '了解', '事物', '真正', '含义', '的', '秘密', '取决于', '如何', '将', '其', '与', '我们', '所', '了解', '的', '事物', '相', '联系', '。']
--------------------------------------------------
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
--------------------------------------------------
  (0, 20)	0.2182178902359924
  (0, 2)	0.2182178902359924
  (0, 27)	0.2182178902359924
  (0, 19)	0.2182178902359924
  (0, 24)	0.2182178902359924
  (0, 14)	0.2182178902359924
  (0, 31)	

## 特征处理
### 归一化处理--MinMaxScaler
### 标准化处理--StandardScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def mm():
    """
    归一化处理
    :return: NOne
    """
    # 归一化容易受极值的影响,默认feature_range为(0,1)
    mm = MinMaxScaler(feature_range=(-1, 1))
    mm1= MinMaxScaler(feature_range=(0, 1))

    data = mm.fit_transform([[90, 2, 10, 40], [60, 4, 15, 45], [75, 3, 13, 46]])
    data1 = mm1.fit_transform([[90, 2, 10, 40], [60, 4, 15, 45], [75, 3, 13, 46]])

    print('-1--1范围内归一化处理结果\n',data)
    print('-'*50)
    print('0--1范围内归一化处理结果\n',data1)

    return None

mm()

归一化处理总结:<br>
归一化易受极值影响,但计算简便快速<br>
归一化在数据量较小,不存在极端数据情况时效果较好<br>
归一化处理值计算：x'=(x-min)/max-min--(0,1)之间的范围<br>
范围转换扩展计算：x''=x'*(mx-mi)+mi,得到(mi,mx)的新的区间范围<br>
当然这些都可以通过定义时,feature_range来直接设置

In [49]:
from sklearn.preprocessing import StandardScaler
def stand():
    """
    标准化缩放，不是标准正太分布，只均值为0，方差为1的分布
    :return:
    """
    # 创建标准化类
    std = StandardScaler()
    # 进行数据转换
    data = std.fit_transform([[1., -1., 3.], [2., 4., 2.], [4., 6., -1.]])

    print('标准化后所得数据\n',data)
    print('-'*50)

    # 均值
    print('均值\n',std.mean_)
    print('-'*50)
    # 方差
    print('方差\n',std.var_)
    print('-'*50)
    # 样本数
    print('样本数\n',std.n_samples_seen_)
    print('-' * 50)

    # 对处理后的数据再次标准化转换，反过来验证
    data1 = std.fit_transform([[-1.06904497, -1.35873244, 0.98058068],
                               [-0.26726124, 0.33968311, 0.39223227],
                               [1.33630621, 1.01904933, -1.37281295]])
    # 查看处理后的特征的期望与方差是否符合预期
    # 均值
    print('均值\n',std.mean_)
    print('-'*50)
    # 方差
    print('方差\n',std.var_)
    print('-'*50)
    # 样本数
    print('样本数\n',std.n_samples_seen_)
    return None

stand()

标准化后所得数据
 [[-1.06904497 -1.35873244  0.98058068]
 [-0.26726124  0.33968311  0.39223227]
 [ 1.33630621  1.01904933 -1.37281295]]
--------------------------------------------------
均值
 [2.33333333 3.         1.33333333]
--------------------------------------------------
方差
 [1.55555556 8.66666667 2.88888889]
--------------------------------------------------
样本数
 3
--------------------------------------------------
均值
 [0. 0. 0.]
--------------------------------------------------
方差
 [1.         1.         1.00000001]
--------------------------------------------------
样本数
 3


标准化缩放总结:<br>
计算公式: x'=(x-mean)/σ,mean为均值,σ为标准差<br>
标准化是指,把原始数据通过处理后得到期望为0,方差为1的标准化分布数据集,并不是标准正态分布<br>
相对于归一化,并不受极值的影响,更适合大规模的数据

## 数据处理中缺失值的处理

In [57]:
from sklearn.impute import SimpleImputer
import numpy
#下面是填补，针对删除，可以用pd和np
def im():
    """
    缺失值处理
    :return:None
    """
    # NaN, nan,缺失值必须是这种形式，如果是？号(或者其他符号)，就要replace换成这种
    # 创建SimpleImputer类
    # 其中参数,missing_values指定要替换的数据,strategy指定替换后的数据
    # 可以传入字符串表示的计算方法,如['mean', 'median', 'most_frequent', 'constant']
    im = SimpleImputer(missing_values=np.nan, strategy='mean')
    data = im.fit_transform([[1, 2], [np.nan, 3], [7, 6], [3, 2]])
    print(data)

    return None

im()

[[1.         2.        ]
 [3.66666667 3.        ]
 [7.         6.        ]
 [3.         2.        ]]


## 降维处理
降维直接说就是减少特征数<br>
从而可以提高模型训练速度

In [58]:
from sklearn.feature_selection import VarianceThreshold
def var():
    """
    特征选择-删除低方差的特征
    :return: None
    """
    #默认只删除方差为0,threshold是方差阈值，删除比这个值小的那些特征
    # 删除方差较小的特征值,因为大量方差较小的特征,说明这一特征的特点都相差不大,没有训练区分的必要
    var = VarianceThreshold(threshold=1)

    data = var.fit_transform([[0, 2, 0, 3],
                              [0, 1, 4, 3],
                              [0, 1, 1, 3]])

    print(data)
    # 获得剩余的特征的列编号
    print('The surport is %s' % var.get_support(True))
    return None

var()




[[0]
 [4]
 [1]]
The surport is [2]
